<a href="https://colab.research.google.com/github/FernandaAroca/APRENDIZADO-DE-MAQUINA-I---Ci-ncias-de-Dados-/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_ML7_ex_projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***Título do seu projeto***
---

In [1]:
#@title Identificação do Grupo

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = "'10730786, Fernanda G Aroca'" #@param {type:"string"}
Aluno2 = '1115677, Daniel Henrique' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



# **Apresentação**

# Problema

*Optamos por prever a inadimplência dos clientes classificação binária, um verdadeiro desafio para bancos e fintechs, que ajuda a minimizar prejuízos e aprimorar as decisões de crédito. A fim de atender a necessidade de dados originais, criamos programaticamente um conjunto de dados inicial com 2000 entradas, repletas de variáveis comuns idade, renda, pontuação, saldo, tempo de relacionamento, etc. e uma etiqueta de inadimplência simulada, oque posibilita o treinamento e a avaliação dos modelos.*



# Referencial Teórico

*O trabalho apoia-se nos princípios básicos do aprendizado supervisionado para classificação: definição da variável alvo, pré-processamento (escalonamento e codificação), engenharia de features, treino/validação e avaliação por métricas como precisão, recall, F1 e AUC. Para comparação de modelos seguimos a orientação prática de testar um modelo linear interpretável (regressão logística) e um ensemble mais flexível (Random Forest), avaliando trade-offs entre desempenho e interpretabilidade.
Referências rápidas

Material da disciplina — AULA 7: Seleção de modelos, classificação e preparação de dados. Texto de Apoio 7.
Scikit-learn — Choosing the Right Estimator (guia prático).


Scikit-learn — Classifier comparison (exemplos de comparação entre classificadores).


Kaggle — Time Series (leitura recomendada sobre engenharia de features; complementar).


Peixeiro, Time Series Forecasting in Python (referência sobre modelos estatísticos de séries temporais).

*

# Metodologia

*Geramos uma base original de 2000 registros, tratando o desafio como classificação binária, visando prever inadimplência. Variáveis comuns de crédito foram empregadas — idade, renda, score, tempo de relacionamento, número de produtos, saldo, salário estimado, indicador de atividade e posse de cartão — em conjunto com a variável alvo default. Os dados foram divididos de maneira estratificada em treino (70%) e teste (30%), onde no pré-processamento as variáveis numéricas foram escalonadas e o one-hot encoding aplicado nas categóricas tudo organizado num pipeline mantendo a reprodutibilidade. Treinamos dois modelos, um de regressão logística — interpretável, uma base, e outro Random Forest para relações não lineares comparando o desempenho por accuracy, precision, recall, F1 e ROC AUC, ademais examinamos a matriz de confusão e importâncias de variáveis, tentando entender os resultados.*



# **Implementação**

# Base de Dados

*A base de dados foi criada especificamente para o projeto, com 2.000 registros simulando informações de clientes, como idade, renda, score de crédito, saldo e atividade da conta. A variável alvo é default (inadimplência). Após a geração, os dados foram divididos em treino e teste (70/30) e passaram por pré-processamento: padronização das variáveis numéricas e one-hot encoding das categóricas, tudo organizado em um pipeline para garantir consistência.*




In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

RND = 42
np.random.seed(RND)

n = 2000

age = np.random.randint(18, 76, size=n)
annual_income = np.random.lognormal(mean=10.5, sigma=0.9, size=n)
credit_score = np.clip((np.random.normal(loc=650, scale=70, size=n)).astype(int), 300, 850)
tenure_years = np.random.randint(0, 11, size=n)
num_products = np.random.choice([1,2,3,4], size=n, p=[0.5,0.3,0.15,0.05])
balance = np.random.exponential(scale=20000, size=n)
is_active = np.random.binomial(1, 0.7, size=n)
has_cr_card = np.random.binomial(1, 0.8, size=n)
estimated_salary = annual_income * np.random.uniform(0.85, 1.15, size=n)
geography = np.random.choice(['Brazil', 'Portugal', 'Angola'], size=n, p=[0.85,0.1,0.05])

score = (
    -0.008 * credit_score +
    0.00002 * balance -
    -0.12 * tenure_years -
    -0.6 * is_active -
    -0.3 * has_cr_card -
    -0.2 * num_products +
    0.00001 * annual_income +
    np.where(geography=='Brazil', 0.0, 0.15 * (geography=='Portugal'))
)

prob_default = 1 / (1 + np.exp(-score))
prob_default = np.clip(prob_default, 0.01, 0.99)
default = (np.random.rand(n) < prob_default).astype(int)


df = pd.DataFrame({
    'age': age,
    'annual_income': annual_income,
    'credit_score': credit_score,
    'tenure_years': tenure_years,
    'num_products': num_products,
    'balance': balance,
    'is_active': is_active,
    'has_cr_card': has_cr_card,
    'estimated_salary': estimated_salary,
    'geography': geography,
    'default': default
})


X = df.drop(columns=['default'])
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=RND
)


numeric_features = [
    'age','annual_income','credit_score',
    'tenure_years','num_products','balance',
    'estimated_salary'
]

categorical_features = ['is_active','has_cr_card','geography']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

## **Modelo 1**

*O primeiro modelo utilizado foi a Regressão Logística, escolhida por ser simples, interpretável e servir como baseline em problemas de classificação binária. Ele estima a probabilidade de inadimplência com base em uma combinação linear das variáveis.*


### **Modelo 1:** Preparação dos Dados


In [ ]:

X_train_lr = X_train.copy()
X_test_lr = X_test.copy()

### **Modelo 1:** Modelo


In [ ]:
from sklearn.linear_model import LogisticRegression

modelo1 = Pipeline(steps=[
    ("preprocessamento", preprocessor),
    ("classificador", LogisticRegression(max_iter=1000, random_state=42))
])

modelo1.fit(X_train_lr, y_train)
y_pred_lr = modelo1.predict(X_test_lr)
y_proba_lr = modelo1.predict_proba(X_test_lr)[:, 1]

### **Modelo 1:** Resultados


In [ ]:
print("Resultados – Modelo 1 (Regressão Logística)")
print("Acurácia:", accuracy_score(y_test, y_pred_lr))
print("Precisão:", precision_score(y_test, y_pred_lr, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_lr, zero_division=0))
print("F1:", f1_score(y_test, y_pred_lr, zero_division=0))
print("ROC AUC:", roc_auc_score(y_test, y_proba_lr))

print("\nMatriz de confusão:")
print(confusion_matrix(y_test, y_pred_lr))

## **Modelo 2**

*O segundo modelo empregado foi o Random Forest, um ensemble de múltiplas árvores de decisão capaz de captar relações não lineares e interações entre variáveis.*


### **Modelo 2:** Preparação dos Dados


In [ ]:
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()

### **Modelo 2:** Modelo


In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo2 = Pipeline(steps=[
    ("preprocessamento", preprocessor),
    ("classificador", RandomForestClassifier(n_estimators=200, random_state=42))
])

modelo2.fit(X_train_rf, y_train)
y_pred_rf = modelo2.predict(X_test_rf)
y_proba_rf = modelo2.predict_proba(X_test_rf)[:, 1]

### **Modelo 2:** Resultados


In [ ]:
print("Resultados – Modelo 2 (Random Forest)")
print("Acurácia:", accuracy_score(y_test, y_pred_rf))
print("Precisão:", precision_score(y_test, y_pred_rf, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_rf, zero_division=0))
print("F1:", f1_score(y_test, y_pred_rf, zero_division=0))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))

print("\nMatriz de confusão:")
print(confusion_matrix(y_test, y_pred_rf))

print("\nImportância das variáveis (top 10):")
import numpy as np
import pandas as pd

# Recuperando nomes das features transformadas
pre = modelo2.named_steps["preprocessamento"]
ohe = pre.named_transformers_["cat"].named_steps["onehot"]

cat_names = []
for col, cats in zip(categorical_features, ohe.categories_):
    for c in cats:
        cat_names.append(f"{col}_{c}")

feature_names = numeric_features + cat_names

importances = modelo2.named_steps["classificador"].feature_importances_

imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

print(imp_df.head(10))

# **Conclusão**

*Ao longo do projeto, desenvolvi uma solução de aprendizado supervisionado para prever inadimplência a partir de uma base original criada especificamente para esse estudo. Após gerar os dados e aplicar o pré-processamento adequado, treinei dois modelos — Regressão Logística e Random Forest — buscando comparar suas performances. Apesar da alta acurácia obtida por ambos, os resultados mostraram que nenhum dos modelos conseguiu identificar os casos de inadimplência, principalmente devido ao forte desbalanceamento da base, em que a classe positiva representa menos de 3% dos registros. Isso reforça o que é apontado no referencial teórico: em problemas com classes desbalanceadas, acurácia não é uma métrica confiável, sendo mais adequado focar em recall, F1 e PR-AUC.

Mesmo com essa limitação, o trabalho permitiu compreender bem o fluxo completo de criação de dados, preparação, modelagem e avaliação. Como continuação natural, melhorias futuras incluiriam técnicas de balanceamento (como SMOTE), ajuste de thresholds, validação cruzada e tuning de hiperparâmetros, além da possibilidade de gerar mais exemplos positivos para enriquecer o aprendizado dos modelos. Assim, o estudo demonstra com clareza como os modelos se comportam em cenários de risco de crédito e como ajustes adicionais são fundamentais para melhorar a detecção de inadimplentes.*



# **Referências**

*Material da disciplina — AULA 7. Seleção de modelos, classificação e preparação de dados.

Documentação do Scikit-learn – Guia de escolha de modelos.

Documentação do Scikit-learn – Comparação de classificadores.*

---

In [ ]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos.
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados.
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta.
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [2]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

NameError: name 'Completo' is not defined